In [1]:
from setuptools import find_packages

In [2]:
import torch
from torch import nn

In [3]:
import pprint

In [4]:
import json
import random
import numpy as np
import collections
from collections import defaultdict
import itertools

In [2]:
find_packages(exclude=('configs', 'tools', 'demo'))

['mmcv_custom',
 'mmdet',
 'mmcv_custom.runner',
 'mmdet.apis',
 'mmdet.core',
 'mmdet.datasets',
 'mmdet.utils',
 'mmdet.models',
 'mmdet.core.visualization',
 'mmdet.core.post_processing',
 'mmdet.core.utils',
 'mmdet.core.mask',
 'mmdet.core.evaluation',
 'mmdet.core.anchor',
 'mmdet.core.export',
 'mmdet.core.bbox',
 'mmdet.core.bbox.match_costs',
 'mmdet.core.bbox.iou_calculators',
 'mmdet.core.bbox.assigners',
 'mmdet.core.bbox.samplers',
 'mmdet.core.bbox.coder',
 'mmdet.datasets.pipelines',
 'mmdet.datasets.api_wrappers',
 'mmdet.datasets.samplers',
 'mmdet.models.losses',
 'mmdet.models.dense_heads',
 'mmdet.models.necks',
 'mmdet.models.utils',
 'mmdet.models.backbones',
 'mmdet.models.detectors',
 'mmdet.models.roi_heads',
 'mmdet.models.roi_heads.shared_heads',
 'mmdet.models.roi_heads.roi_extractors',
 'mmdet.models.roi_heads.bbox_heads',
 'mmdet.models.roi_heads.mask_heads']

In [3]:
!pwd

/Users/yupeng/Documents/2021summer/SummerIntern/QueryInst


In [4]:
find_packages(exclude=('configs', 'tools'))  

['mmcv_custom',
 'mmdet',
 'mmcv_custom.runner',
 'mmdet.apis',
 'mmdet.core',
 'mmdet.datasets',
 'mmdet.utils',
 'mmdet.models',
 'mmdet.core.visualization',
 'mmdet.core.post_processing',
 'mmdet.core.utils',
 'mmdet.core.mask',
 'mmdet.core.evaluation',
 'mmdet.core.anchor',
 'mmdet.core.export',
 'mmdet.core.bbox',
 'mmdet.core.bbox.match_costs',
 'mmdet.core.bbox.iou_calculators',
 'mmdet.core.bbox.assigners',
 'mmdet.core.bbox.samplers',
 'mmdet.core.bbox.coder',
 'mmdet.datasets.pipelines',
 'mmdet.datasets.api_wrappers',
 'mmdet.datasets.samplers',
 'mmdet.models.losses',
 'mmdet.models.dense_heads',
 'mmdet.models.necks',
 'mmdet.models.utils',
 'mmdet.models.backbones',
 'mmdet.models.detectors',
 'mmdet.models.roi_heads',
 'mmdet.models.roi_heads.shared_heads',
 'mmdet.models.roi_heads.roi_extractors',
 'mmdet.models.roi_heads.bbox_heads',
 'mmdet.models.roi_heads.mask_heads']

In [6]:
def parse_requirements(fname='requirements.txt', with_version=True):
    """Parse the package dependencies listed in a requirements file but strips
    specific versioning information.

    Args:
        fname (str): path to requirements file
        with_version (bool, default=False): if True include version specs

    Returns:
        List[str]: list of requirements items

    CommandLine:
        python -c "import setup; print(setup.parse_requirements())"
    """
    import sys
    from os.path import exists
    import re
    require_fpath = fname

    def parse_line(line):
        """Parse information from a line in a requirements text file."""
        if line.startswith('-r '):
            # Allow specifying requirements in other files
            target = line.split(' ')[1]
            for info in parse_require_file(target):
                yield info
        else:
            info = {'line': line}
            if line.startswith('-e '):
                info['package'] = line.split('#egg=')[1]
            elif '@git+' in line:
                info['package'] = line
            else:
                # Remove versioning from the package
                pat = '(' + '|'.join(['>=', '==', '>']) + ')'
                parts = re.split(pat, line, maxsplit=1)
                parts = [p.strip() for p in parts]

                info['package'] = parts[0]
                if len(parts) > 1:
                    op, rest = parts[1:]
                    if ';' in rest:
                        # Handle platform specific dependencies
                        # http://setuptools.readthedocs.io/en/latest/setuptools.html#declaring-platform-specific-dependencies
                        version, platform_deps = map(str.strip,
                                                     rest.split(';'))
                        info['platform_deps'] = platform_deps
                    else:
                        version = rest  # NOQA
                    info['version'] = (op, version)
            yield info

    def parse_require_file(fpath):
        with open(fpath, 'r') as f:
            for line in f.readlines():
                line = line.strip()
                if line and not line.startswith('#'):
                    for info in parse_line(line):
                        yield info

    def gen_packages_items():
        if exists(require_fpath):
            for info in parse_require_file(require_fpath):
                parts = [info['package']]
                if with_version and 'version' in info:
                    parts.extend(info['version'])
                if not sys.version.startswith('3.4'):
                    # apparently package_deps are broken in 3.4
                    platform_deps = info.get('platform_deps')
                    if platform_deps is not None:
                        parts.append(';' + platform_deps)
                item = ''.join(parts)
                yield item

    packages = list(gen_packages_items())
    return packages

In [7]:
parse_requirements('requirements/build.txt')

['cython', 'numpy']

In [8]:
parse_requirements('requirements/tests.txt')

['asynctest',
 'codecov',
 'flake8',
 'interrogate',
 'isort==4.3.21',
 'kwarray',
 'onnx==1.7.0',
 'onnxruntime==1.5.1',
 'pytest',
 'ubelt',
 'xdoctest>=0.10.0',
 'yapf']

In [10]:
parse_requirements('requirements/runtime.txt'),

(['matplotlib',
  'numpy',
  'pycocotools; platform_system=="Linux"',
  'pycocotools-windows; platform_system=="Windows"',
  'six',
  'terminaltables',
  'timm'],)

In [13]:
init_proposal_bboxes = nn.Embedding(10, 4)
res = nn.init.constant_(init_proposal_bboxes.weight[:, :2], 0.5)
res

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]], grad_fn=<AsStridedBackward>)

In [19]:
nn.init.constant_(init_proposal_bboxes.weight[:, 2:], 1)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], grad_fn=<AsStridedBackward>)

In [20]:
init_proposal_bboxes.weight

Parameter containing:
tensor([[0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000],
        [0.5000, 0.5000, 1.0000, 1.0000]], requires_grad=True)

In [23]:
a = torch.zeros((2, 10, 4))
a.shape

torch.Size([2, 10, 4])

In [24]:
len(a)

2

In [26]:
b = [a[i] for i in range(len(a))]
b

[tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]),
 tensor([[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]])]

In [4]:
def bbox2roi(bbox_list):
    """Convert a list of bboxes to roi format.

    Args:
        bbox_list (list[Tensor]): a list of bboxes corresponding to a batch
            of images.

    Returns:
        Tensor: shape (n, 5), [batch_ind, x1, y1, x2, y2]
    """
    rois_list = []
    for img_id, bboxes in enumerate(bbox_list):
        # bboxes.shape: (num_proposals, 4)
        if bboxes.size(0) > 0:
            img_inds = bboxes.new_full((bboxes.size(0), 1), img_id)
            rois = torch.cat([img_inds, bboxes[:, :4]], dim=-1)
        else:
            rois = bboxes.new_zeros((0, 5))
        rois_list.append(rois)
    rois = torch.cat(rois_list, 0)
    return rois

In [33]:
c = torch.rand((4, 2, 4))
d = bbox2roi(c)
d

tensor([[0.0000, 0.4140, 0.7233, 0.4780, 0.1765],
        [0.0000, 0.8594, 0.4138, 0.2550, 0.2919],
        [1.0000, 0.1803, 0.6177, 0.8708, 0.6756],
        [1.0000, 0.6536, 0.2452, 0.3715, 0.2674],
        [2.0000, 0.1102, 0.4175, 0.5090, 0.7105],
        [2.0000, 0.4338, 0.6390, 0.5860, 0.4554],
        [3.0000, 0.8406, 0.4782, 0.0737, 0.2856],
        [3.0000, 0.0529, 0.1080, 0.3798, 0.3995]])

In [31]:
d.shape

torch.Size([8, 5])

In [34]:
c

tensor([[[0.4140, 0.7233, 0.4780, 0.1765],
         [0.8594, 0.4138, 0.2550, 0.2919]],

        [[0.1803, 0.6177, 0.8708, 0.6756],
         [0.6536, 0.2452, 0.3715, 0.2674]],

        [[0.1102, 0.4175, 0.5090, 0.7105],
         [0.4338, 0.6390, 0.5860, 0.4554]],

        [[0.8406, 0.4782, 0.0737, 0.2856],
         [0.0529, 0.1080, 0.3798, 0.3995]]])

In [16]:
proposal_boxes = torch.rand(4, 2, 4)
proposal_list = [proposal_boxes[i] for i in range(len(proposal_boxes))]
rois = bbox2roi(proposal_list)

In [17]:
rois

tensor([[0.0000, 0.8674, 0.6879, 0.2520, 0.3844],
        [0.0000, 0.0416, 0.9977, 0.4560, 0.3463],
        [1.0000, 0.2048, 0.4977, 0.6467, 0.9364],
        [1.0000, 0.1216, 0.4102, 0.1659, 0.1015],
        [2.0000, 0.2531, 0.7194, 0.8648, 0.3340],
        [2.0000, 0.2727, 0.6682, 0.8327, 0.1480],
        [3.0000, 0.5849, 0.6169, 0.1098, 0.8604],
        [3.0000, 0.8024, 0.9218, 0.0160, 0.8042]])

In [5]:
in_channels = [2, 3, 5, 7]
scales = [340, 170, 84, 43]
inputs = [torch.rand(1, c, s, s) for c, s in zip(in_channels, scales)]
inputs[0].shape

torch.Size([1, 2, 340, 340])

In [6]:
inputs[1].shape

torch.Size([1, 3, 170, 170])

In [8]:
# imgs_whwh.append(imgs[0].new_tensor([[w, h, w, h]]))
img_0 = torch.rand((2, 3, 3, 3))
img_0

tensor([[[[0.1859, 0.4968, 0.5601],
          [0.1935, 0.3251, 0.7271],
          [0.2630, 0.7858, 0.9863]],

         [[0.4258, 0.5859, 0.3402],
          [0.0410, 0.3063, 0.6568],
          [0.9656, 0.2459, 0.9027]],

         [[0.5308, 0.5463, 0.0868],
          [0.5445, 0.8437, 0.9497],
          [0.3916, 0.1916, 0.3441]]],


        [[[0.0472, 0.4806, 0.7984],
          [0.2977, 0.9824, 0.7561],
          [0.5655, 0.2376, 0.6125]],

         [[0.8725, 0.2155, 0.0750],
          [0.0554, 0.6008, 0.6296],
          [0.3653, 0.3989, 0.5753]],

         [[0.5374, 0.4930, 0.4973],
          [0.5730, 0.6311, 0.8633],
          [0.9826, 0.0937, 0.3937]]]])

In [10]:
img_0.new_tensor([[2, 2, 2, 2]])

tensor([[2., 2., 2., 2.]])

In [13]:
imgs_whwh = torch.rand((2, 1, 4))
proposals = torch.rand((3, 4))
proposals

tensor([[0.1005, 0.6047, 0.4954, 0.0234],
        [0.5817, 0.8683, 0.5542, 0.7398],
        [0.0536, 0.0303, 0.5816, 0.9956]])

In [14]:
imgs_whwh

tensor([[[0.5256, 0.1174, 0.3877, 0.4371]],

        [[0.2512, 0.5584, 0.2768, 0.0615]]])

In [16]:
final_proposals = proposals * imgs_whwh

In [17]:
final_proposals.shape

torch.Size([2, 3, 4])

In [18]:
final_proposals

tensor([[[0.0528, 0.0710, 0.1921, 0.0102],
         [0.3057, 0.1020, 0.2149, 0.3234],
         [0.0281, 0.0036, 0.2255, 0.4352]],

        [[0.0253, 0.3377, 0.1372, 0.0014],
         [0.1461, 0.4848, 0.1534, 0.0455],
         [0.0134, 0.0169, 0.1610, 0.0612]]])

In [22]:
num_proposals = 3
proposal_feature_channel = 3
init_proposal_features = nn.Embedding(num_proposals, proposal_feature_channel)
init_proposal_features.weight

Parameter containing:
tensor([[-0.4601,  1.9385, -1.3533],
        [-0.0058, -1.2428,  0.0770],
        [-0.5888, -0.8354, -1.3720]], requires_grad=True)

In [25]:
init_proposal_features_weight = init_proposal_features.weight.clone()
init_proposal_features_weight

tensor([[-0.4601,  1.9385, -1.3533],
        [-0.0058, -1.2428,  0.0770],
        [-0.5888, -0.8354, -1.3720]], grad_fn=<CloneBackward>)

In [26]:
init_proposal_features_weight[None]

tensor([[[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]]], grad_fn=<UnsqueezeBackward0>)

In [27]:
init_proposal_features_weight[]

SyntaxError: invalid syntax (<ipython-input-27-bdf5fa19dc76>, line 1)

In [28]:
init_proposal_features_weight[1]

tensor([-0.0058, -1.2428,  0.0770], grad_fn=<SelectBackward>)

In [29]:
init_proposal_features_weight.size()

torch.Size([3, 3])

In [30]:
*init_proposal_features_weight.size()

SyntaxError: can't use starred expression here (<ipython-input-30-8e71f608340f>, line 1)

In [31]:
num_imgs = 2
init_proposal_features_weight[None].expand(num_imgs, *init_proposal_features_weight.size())

tensor([[[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]],

        [[-0.4601,  1.9385, -1.3533],
         [-0.0058, -1.2428,  0.0770],
         [-0.5888, -0.8354, -1.3720]]], grad_fn=<ExpandBackward>)

In [32]:
init_proposal_features_weight[None].expand(num_imgs, *init_proposal_features_weight.size()).size()

torch.Size([2, 3, 3])

In [7]:
featmap_strides = [4, 8, 16, 32]
len(featmap_strides)

4

In [42]:
def map_roi_levels(rois, num_levels):
    """Map rois to corresponding feature levels by scales.

    - scale < finest_scale * 2: level 0
    - finest_scale * 2 <= scale < finest_scale * 4: level 1
    - finest_scale * 4 <= scale < finest_scale * 8: level 2
    - scale >= finest_scale * 8: level 3

    Args:
        rois (Tensor): Input RoIs, shape (k, 5).
        num_levels (int): Total level number.

    Returns:
        Tensor: Level index (0-based) of each RoI, shape (k, )
    """
    # rois: (batch * num_proposals, 5)
    scale = torch.sqrt(
        (rois[:, 3] - rois[:, 1]) * (rois[:, 4] - rois[:, 2]))
    print(scale)
    target_lvls = torch.floor(torch.log2(scale / 56 + 1e-6))
    print(target_lvls)
    target_lvls = target_lvls.clamp(min=0, max=num_levels - 1).long()
    return target_lvls

In [18]:
rois

tensor([[0.0000, 0.8674, 0.6879, 0.2520, 0.3844],
        [0.0000, 0.0416, 0.9977, 0.4560, 0.3463],
        [1.0000, 0.2048, 0.4977, 0.6467, 0.9364],
        [1.0000, 0.1216, 0.4102, 0.1659, 0.1015],
        [2.0000, 0.2531, 0.7194, 0.8648, 0.3340],
        [2.0000, 0.2727, 0.6682, 0.8327, 0.1480],
        [3.0000, 0.5849, 0.6169, 0.1098, 0.8604],
        [3.0000, 0.8024, 0.9218, 0.0160, 0.8042]])

In [43]:
map_roi_res = map_roi_levels(rois, 4)

tensor([0.4322,    nan, 0.4403,    nan,    nan,    nan,    nan, 0.3041])
tensor([-8., nan, -7., nan, nan, nan, nan, -8.])


In [44]:
map_roi_res.shape

torch.Size([8])

In [45]:
map_roi_res

tensor([                   0, -9223372036854775808,                    0,
        -9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808,                    0])

In [32]:
map_roi_res == 1

tensor([False, False, False, False, False, False, False, False])

In [34]:
all_one = torch.ones((4, 2, 4))
all_one_res = bbox2roi(all_one)
all_one_res

tensor([[0., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [2., 1., 1., 1., 1.],
        [2., 1., 1., 1., 1.],
        [3., 1., 1., 1., 1.],
        [3., 1., 1., 1., 1.]])

In [36]:
map_roi_levels(all_one_res, 4)

tensor([0., 0., 0., 0., 0., 0., 0., 0.])


tensor([0, 0, 0, 0, 0, 0, 0, 0])

In [37]:
torch.floor(torch.rand(3))

tensor([0., 0., 0.])

In [38]:
torch.floor(torch.ones(3))

tensor([1., 1., 1.])

In [46]:
test_roi = bbox2roi(torch.rand((4, 2, 4)))
test_roi

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [0.0000, 0.7373, 0.4391, 0.9600, 0.6475],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [1.0000, 0.1370, 0.0319, 0.7754, 0.8904],
        [2.0000, 0.2996, 0.0309, 0.0516, 0.9780],
        [2.0000, 0.2541, 0.1225, 0.8034, 0.4431],
        [3.0000, 0.9790, 0.4394, 0.4774, 0.2575],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [47]:
torch.sqrt((test_roi[:, 3] - test_roi[:, 1]) * (test_roi[:, 4] - test_roi[:, 2]))

tensor([   nan, 0.2154, 0.6051, 0.7404,    nan, 0.4196, 0.3020,    nan])

In [48]:
(test_roi[:, 3] - test_roi[:, 1]) * (test_roi[:, 4] - test_roi[:, 2])

tensor([-0.2045,  0.0464,  0.3662,  0.5481, -0.2349,  0.1761,  0.0912, -0.1586])

In [49]:
test_roi[:, 3] - test_roi[:, 1]

tensor([-0.4999,  0.2227, -0.6760,  0.6384, -0.2480,  0.5494, -0.5015,  0.7376])

In [50]:
test_roi[:, 3]

tensor([0.2630, 0.9600, 0.0100, 0.7754, 0.0516, 0.8034, 0.4774, 0.9170])

In [57]:
mask = map_roi_res == 0
inds = mask.nonzero(as_tuple=False).squeeze(1)
inds

tensor([0, 2, 7])

In [59]:
mask

tensor([ True, False,  True, False, False, False, False,  True])

In [60]:
map_roi_res

tensor([                   0, -9223372036854775808,                    0,
        -9223372036854775808, -9223372036854775808, -9223372036854775808,
        -9223372036854775808,                    0])

In [58]:
mask.nonzero(as_tuple=False)

tensor([[0],
        [2],
        [7]])

In [61]:
test_roi

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [0.0000, 0.7373, 0.4391, 0.9600, 0.6475],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [1.0000, 0.1370, 0.0319, 0.7754, 0.8904],
        [2.0000, 0.2996, 0.0309, 0.0516, 0.9780],
        [2.0000, 0.2541, 0.1225, 0.8034, 0.4431],
        [3.0000, 0.9790, 0.4394, 0.4774, 0.2575],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [62]:
test_roi[inds]

tensor([[0.0000, 0.7629, 0.3841, 0.2630, 0.7933],
        [1.0000, 0.6860, 0.6027, 0.0100, 0.0610],
        [3.0000, 0.1794, 0.8235, 0.9170, 0.6085]])

In [63]:
inds

tensor([0, 2, 7])

In [64]:
proposal_feat = torch.rand((2, 2, 3))
proposal_feat

tensor([[[0.6784, 0.7932, 0.5145],
         [0.0992, 0.0859, 0.5313]],

        [[0.4867, 0.9737, 0.1892],
         [0.3729, 0.2131, 0.0048]]])

In [65]:
proposal_feat.permute(1, 0, 2)

tensor([[[0.6784, 0.7932, 0.5145],
         [0.4867, 0.9737, 0.1892]],

        [[0.0992, 0.0859, 0.5313],
         [0.3729, 0.2131, 0.0048]]])

In [4]:
test_new_zeros = torch.rand((2, 3, 4))
test_new_zeros_1 = torch.rand((2, 4))
gen = test_new_zeros.new_zeros(len(test_new_zeros))
gen_1 = test_new_zeros_1.new_zeros(len(test_new_zeros_1))

In [5]:
gen.shape

torch.Size([2])

In [7]:
gen_1.shape

torch.Size([2])

In [8]:
test_new_zeros_1[0]

tensor([0.8472, 0.7459, 0.4239, 0.9840])

In [9]:
test_new_zeros.shape

torch.Size([2, 3, 4])

In [10]:
test_new_zeros_1 * 4

tensor([[3.3887, 2.9836, 1.6955, 3.9359],
        [2.0148, 0.6900, 0.7875, 2.2636]])

In [12]:
label = torch.rand(3)
inds = torch.stack((label, label + 1, label + 2, label + 3), 1)
inds.shape

torch.Size([3, 4])

In [13]:
label

tensor([0.0178, 0.2046, 0.7053])

In [14]:
inds

tensor([[0.0178, 1.0178, 2.0178, 3.0178],
        [0.2046, 1.2046, 2.2046, 3.2046],
        [0.7053, 1.7053, 2.7053, 3.7053]])

In [16]:
num_rois = 1
num_proposals = 3
pos_is_gts_ = torch.ones(num_proposals)
pos_keep = 1 - pos_is_gts_  # (num_proposals,) 实际上全是1
keep_inds = pos_is_gts_.new_ones(num_rois)  # (num_rois, )
keep_inds[:len(pos_is_gts_)] = pos_keep

RuntimeError: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1].  Tensor sizes: [3]

In [4]:
[None] * 1

[None]

In [5]:
get_res_none = [None] * 1
type(get_res_none[0])

NoneType

In [5]:
dataset = json.load(open('../vis2021/results.json', 'r'))

In [7]:
dataset[0]['video_id']

1

In [8]:
len(dataset)

3244

In [9]:
dataset[0]['segmentations']

[None,
 {'size': [720, 1280],
  'counts': 'Xab;5Yf03M2O1N2O1O1O0010O10O100000MQZODPf0<PZOEPf0>O000010O00001O001O001O0O1LmYOJTf06lYOIUf0720001O00hYOJVf09O001O00010O000001O0O2O00[P_?'},
 None,
 None,
 {'size': [720, 1280],
  'counts': 'dah02Vf0;G7M4M2N101N101O1O1N101O001O01O010O100O010O2O1N2O1O0O2O0O1O100O00100O010O00100O01O010O001O0000001O01O01O001O00001O00001N1000001O001O0000001O0O100O2M2M3N2O2N1010O01O1O010O001O01O01O01O01O001O0010O01O01O01O0010O01O00001O001O01O01O00010O000010O02N0001O000000001O000000001O00001O0000001O0000001O00001O001O00001O0000001O00001O00001O001O0000001O00001O00001O000000001O0000000000001O00000000001O001O001O001O001O1O1O001O2N1O1O001O1O1O002N1O1O1O000000QZ_f0'},
 {'size': [720, 1280],
  'counts': 'Y[`33]f000000000000000001O01O01O005K1O2N3M2N2N1O1001O1O0000O100O100O006J1O00100O001O1O001O001O01O0001O000000001O000000000000001O000000001O1O001O1O1O1[[OjNnc0W1o[OmNoc0T1n[OoNQd0R1m[OoNSd0R1j[OQOUd0P1i[OQOWd0o0i[OQOWd0Q1f[OQOYd0c1000000001O000O10000000000000000O101N1O1N2O1

In [10]:
type(dataset)

list

In [11]:
vidIds = [[]]
len(vidIds)

1

In [12]:
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

In [13]:
_isArrayLike([])

True

In [14]:
dict1 = {1: "aaa", 2: "bbb"}
ids = dict1.keys()

In [15]:
ids

dict_keys([1, 2])

In [16]:
for i in ids:
    print(i)

1
2


In [17]:
type((1, 2))

tuple

In [18]:
isinstance((1, 2), list)

False

In [19]:
vid,  frame_id = 1

TypeError: cannot unpack non-iterable int object

In [21]:
random.choice('abc')

'b'

In [22]:
random.choice([1, 2, 3, 4])

3

In [5]:
a = [[1, 2]]
b = np.array(a, dtype=int)

In [6]:
b

array([[1, 2]])

In [9]:
vidToAnns = defaultdict(list)
vidToAnns[1].append({1: 2})
vidToAnns[1].append({1: 3})
vidToAnns[2].append({2: 2})
vidToAnns[2].append({2: 3})

In [13]:
vidIds = [1, 2]
lists = [vidToAnns[vidId] for vidId in vidIds if vidId in vidToAnns]
anns = list(itertools.chain.from_iterable(lists))

In [14]:
lists

[[{1: 2}, {1: 3}], [{2: 2}, {2: 3}]]

In [15]:
anns

[{1: 2}, {1: 3}, {2: 2}, {2: 3}]

In [16]:
img_ids = []
img_ids.append((0, 1))
img_ids.append((0, 2))
img_ids.append((1, 1))
img_ids.append((1, 2))

In [17]:
img_ids[1]

(0, 2)

In [18]:
bboxes = [None, [1,2], None, [3, 4]]

In [19]:
bboxes

[None, [1, 2], None, [3, 4]]

In [20]:
bboxes[1]

[1, 2]

In [21]:
def random_scale(img_scales, mode='range'):
    """Randomly select a scale from a list of scales or scale ranges.

    Args:
        img_scales (list[tuple]): Image scale or scale range.
        mode (str): "range" or "value".

    Returns:
        tuple: Sampled image scale.
    """
    num_scales = len(img_scales)
    if num_scales == 1:  # fixed scale is specified
        img_scale = img_scales[0]
    elif num_scales == 2:  # randomly sample a scale
        if mode == 'range':
            img_scale_long = [max(s) for s in img_scales]
            img_scale_short = [min(s) for s in img_scales]
            long_edge = np.random.randint(
                min(img_scale_long),
                max(img_scale_long) + 1)
            short_edge = np.random.randint(
                min(img_scale_short),
                max(img_scale_short) + 1)
            img_scale = (long_edge, short_edge)
        elif mode == 'value':
            img_scale = img_scales[np.random.randint(num_scales)]
    else:
        if mode != 'value':
            raise ValueError(
                'Only "value" mode supports more than 2 image scales')
        img_scale = img_scales[np.random.randint(num_scales)]
    return img_scale

In [32]:
random_scale([(640, 260), (320, 130)])

(405, 240)

In [33]:
random_scale([(640, 260), (320, 130)])

(517, 185)

In [41]:
def a(x):
    if x==2:
        return 4
    return 6
def b(x):
    if x==1:
        return 2
    return 3
@a
@b
def test_at():
    return 1

In [42]:
test_at()

TypeError: 'int' object is not callable

In [43]:
test_at

6

In [53]:
cfg=dict(
    type='QueryInst',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch')
    )
train_cfg=dict(
        rpn=None,
        )

test_cfg=dict(rpn=None, mask_thr_binary=0.5, nms=dict(type='nms', iou_threshold=0.7))

In [54]:
args = cfg.copy()
default_args = dict(train_cfg=train_cfg, test_cfg=test_cfg)
if default_args is not None:
    for name, value in default_args.items():
        args.setdefault(name, value)

In [55]:
args

{'type': 'QueryInst',
 'pretrained': 'torchvision://resnet50',
 'backbone': {'type': 'ResNet',
  'depth': 50,
  'num_stages': 4,
  'out_indices': (0, 1, 2, 3),
  'frozen_stages': 1,
  'norm_cfg': {'type': 'BN', 'requires_grad': True},
  'norm_eval': True,
  'style': 'pytorch'},
 'train_cfg': {'rpn': None},
 'test_cfg': {'rpn': None,
  'mask_thr_binary': 0.5,
  'nms': {'type': 'nms', 'iou_threshold': 0.7}}}

In [56]:
type(args)

dict

In [4]:
x = torch.rand((2, 3))
x_n = 2
x_split = torch.split(x, x_n, dim=0)

In [6]:
prods = []
for i in range(x_n):
    prod = torch.mm(x_split[i], torch.transpose(x_split[i], 0, 1))
    prods.append(prod)

IndexError: tuple index out of range

In [7]:
res = collections.OrderedDict([('bbox_mAP', 0.469), ('bbox_mAP_50', 0.659), ('bbox_mAP_75', 0.514), ('bbox_mAP_s', 0.302), ('bbox_mAP_m', 0.496), ('bbox_mAP_l', 0.617), ('bbox_mAP_copypaste', '0.469 0.659 0.514 0.302 0.496 0.617'), ('segm_mAP', 0.414), ('segm_mAP_50', 0.638), ('segm_mAP_75', 0.451), ('segm_mAP_s', 0.231), ('segm_mAP_m', 0.443), ('segm_mAP_l', 0.598), ('segm_mAP_copypaste', '0.414 0.638 0.451 0.231 0.443 0.598')])

In [8]:
res

OrderedDict([('bbox_mAP', 0.469),
             ('bbox_mAP_50', 0.659),
             ('bbox_mAP_75', 0.514),
             ('bbox_mAP_s', 0.302),
             ('bbox_mAP_m', 0.496),
             ('bbox_mAP_l', 0.617),
             ('bbox_mAP_copypaste', '0.469 0.659 0.514 0.302 0.496 0.617'),
             ('segm_mAP', 0.414),
             ('segm_mAP_50', 0.638),
             ('segm_mAP_75', 0.451),
             ('segm_mAP_s', 0.231),
             ('segm_mAP_m', 0.443),
             ('segm_mAP_l', 0.598),
             ('segm_mAP_copypaste', '0.414 0.638 0.451 0.231 0.443 0.598')])

In [5]:
cfg = {'dataset_type': 'CocoDataset', 'data_root': 'data/coco/', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'data': {'samples_per_gpu': 2, 'workers_per_gpu': 2, 'train': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_train2017.json', 'img_prefix': 'data/coco/train2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]}, 'val': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_val2017.json', 'img_prefix': 'data/coco/val2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]}, 'test': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_val2017.json', 'img_prefix': 'data/coco/val2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]}}, 'evaluation': {'metric': ['bbox', 'segm']}, 'optimizer': {'type': 'AdamW', 'lr': 2.5e-05, 'weight_decay': 0.0001}, 'optimizer_config': {'grad_clip': {'max_norm': 1, 'norm_type': 2}}, 'lr_config': {'policy': 'step', 'warmup': 'linear', 'warmup_iters': 1000, 'warmup_ratio': 0.001, 'step': [8, 11]}, 'runner': {'type': 'EpochBasedRunner', 'max_epochs': 12}, 'checkpoint_config': {'interval': 1}, 'log_config': {'interval': 50, 'hooks': [{'type': 'TextLoggerHook'}]}, 'custom_hooks': [{'type': 'NumClassCheckHook'}], 'dist_params': {'backend': 'nccl'}, 'log_level': 'INFO', 'load_from': None, 'resume_from': None, 'workflow': [('train', 1)], 'num_stages': 6, 'num_proposals': 100, 'model': {'type': 'QueryInst', 'pretrained': 'torchvision://resnet50', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch'}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'start_level': 0, 'add_extra_convs': 'on_input', 'num_outs': 4}, 'rpn_head': {'type': 'EmbeddingRPNHead', 'num_proposals': 100, 'proposal_feature_channel': 256}, 'roi_head': {'type': 'QueryRoIHead', 'num_stages': 6, 'stage_loss_weights': [1, 1, 1, 1, 1, 1], 'proposal_feature_channel': 256, 'bbox_roi_extractor': {'type': 'SingleRoIExtractor', 'roi_layer': {'type': 'RoIAlign', 'output_size': 7, 'sampling_ratio': 2}, 'out_channels': 256, 'featmap_strides': [4, 8, 16, 32]}, 'mask_roi_extractor': {'type': 'SingleRoIExtractor', 'roi_layer': {'type': 'RoIAlign', 'output_size': 14, 'sampling_ratio': 2}, 'out_channels': 256, 'featmap_strides': [4, 8, 16, 32]}, 'bbox_head': [{'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}], 'mask_head': [{'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}]}, 'train_cfg': {'rpn': None, 'rcnn': [{'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}]}, 'test_cfg': {'rpn': None, 'rcnn': {'max_per_img': 100, 'mask_thr_binary': 0.5, 'nms': {'type': 'nms', 'iou_threshold': 0.7}}}}, 'total_epochs': 12, 'work_dir': './work_dirs/queryinst_r50_fpn_1x_coco', 'gpu_ids': range(0, 1), 'seed': None}

In [7]:
type(cfg)

dict

In [8]:
for key,value in cfg.items():
    print('{key}:{value}'.format(key = key, value = value))

dataset_type:CocoDataset
data_root:data/coco/
img_norm_cfg:{'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}
train_pipeline:[{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]
test_pipeline:[{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']},

In [6]:
cfg1 = {'dataset_type': 'CocoDataset', 'data_root': 'data/coco/', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'AutoAugment', 'policies': [[{'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}], [{'type': 'Resize', 'img_scale': [(400, 1333), (500, 1333), (600, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}, {'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (384, 600), 'allow_negative_crop': True}, {'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'override': True, 'keep_ratio': True}]]}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}], 'data': {'samples_per_gpu': 2, 'workers_per_gpu': 2, 'train': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_train2017.json', 'img_prefix': 'data/coco/train2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'AutoAugment', 'policies': [[{'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}], [{'type': 'Resize', 'img_scale': [(400, 1333), (500, 1333), (600, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}, {'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (384, 600), 'allow_negative_crop': True}, {'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'override': True, 'keep_ratio': True}]]}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]}, 'val': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_val2017.json', 'img_prefix': 'data/coco/val2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]}, 'test': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_val2017.json', 'img_prefix': 'data/coco/val2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img']}]}]}}, 'evaluation': {'metric': ['bbox', 'segm']}, 'optimizer': {'type': 'AdamW', 'lr': 2.5e-05, 'weight_decay': 0.0001}, 'optimizer_config': {'grad_clip': {'max_norm': 1, 'norm_type': 2}}, 'lr_config': {'policy': 'step', 'warmup': 'linear', 'warmup_iters': 1000, 'warmup_ratio': 0.001, 'step': [27, 33]}, 'runner': {'type': 'EpochBasedRunner', 'max_epochs': 36}, 'checkpoint_config': {'interval': 1}, 'log_config': {'interval': 50, 'hooks': [{'type': 'TextLoggerHook'}]}, 'custom_hooks': [{'type': 'NumClassCheckHook'}], 'dist_params': {'backend': 'nccl'}, 'log_level': 'INFO', 'load_from': None, 'resume_from': None, 'workflow': [('train', 1)], 'num_stages': 6, 'num_proposals': 300, 'model': {'type': 'QueryInst', 'pretrained': 'torchvision://resnet50', 'backbone': {'type': 'ResNet', 'depth': 50, 'num_stages': 4, 'out_indices': (0, 1, 2, 3), 'frozen_stages': 1, 'norm_cfg': {'type': 'BN', 'requires_grad': True}, 'norm_eval': True, 'style': 'pytorch'}, 'neck': {'type': 'FPN', 'in_channels': [256, 512, 1024, 2048], 'out_channels': 256, 'start_level': 0, 'add_extra_convs': 'on_input', 'num_outs': 4}, 'rpn_head': {'type': 'EmbeddingRPNHead', 'num_proposals': 300, 'proposal_feature_channel': 256}, 'roi_head': {'type': 'QueryRoIHead', 'num_stages': 6, 'stage_loss_weights': [1, 1, 1, 1, 1, 1], 'proposal_feature_channel': 256, 'bbox_roi_extractor': {'type': 'SingleRoIExtractor', 'roi_layer': {'type': 'RoIAlign', 'output_size': 7, 'sampling_ratio': 2}, 'out_channels': 256, 'featmap_strides': [4, 8, 16, 32]}, 'mask_roi_extractor': {'type': 'SingleRoIExtractor', 'roi_layer': {'type': 'RoIAlign', 'output_size': 14, 'sampling_ratio': 2}, 'out_channels': 256, 'featmap_strides': [4, 8, 16, 32]}, 'bbox_head': [{'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}, {'type': 'DIIHead', 'num_classes': 80, 'num_ffn_fcs': 2, 'num_heads': 8, 'num_cls_fcs': 1, 'num_reg_fcs': 3, 'feedforward_channels': 2048, 'in_channels': 256, 'dropout': 0.0, 'ffn_act_cfg': {'type': 'ReLU', 'inplace': True}, 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 7, 'with_proj': True, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'loss_bbox': {'type': 'L1Loss', 'loss_weight': 5.0}, 'loss_iou': {'type': 'GIoULoss', 'loss_weight': 2.0}, 'loss_cls': {'type': 'FocalLoss', 'use_sigmoid': True, 'gamma': 2.0, 'alpha': 0.25, 'loss_weight': 2.0}, 'bbox_coder': {'type': 'DeltaXYWHBBoxCoder', 'clip_border': False, 'target_means': [0.0, 0.0, 0.0, 0.0], 'target_stds': [0.5, 0.5, 1.0, 1.0]}}], 'mask_head': [{'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}, {'type': 'DynamicMaskHead', 'dynamic_conv_cfg': {'type': 'DynamicConv', 'in_channels': 256, 'feat_channels': 64, 'out_channels': 256, 'input_feat_shape': 14, 'with_proj': False, 'act_cfg': {'type': 'ReLU', 'inplace': True}, 'norm_cfg': {'type': 'LN'}}, 'dropout': 0.0, 'num_convs': 4, 'roi_feat_size': 14, 'in_channels': 256, 'conv_kernel_size': 3, 'conv_out_channels': 256, 'class_agnostic': False, 'norm_cfg': {'type': 'BN'}, 'upsample_cfg': {'type': 'deconv', 'scale_factor': 2}, 'loss_dice': {'type': 'DiceLoss', 'loss_weight': 8.0}}]}, 'train_cfg': {'rpn': None, 'rcnn': [{'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}, {'assigner': {'type': 'HungarianAssigner', 'cls_cost': {'type': 'FocalLossCost', 'weight': 2.0}, 'reg_cost': {'type': 'BBoxL1Cost', 'weight': 5.0}, 'iou_cost': {'type': 'IoUCost', 'iou_mode': 'giou', 'weight': 2.0}}, 'sampler': {'type': 'PseudoSampler'}, 'pos_weight': 1, 'mask_size': 28, 'debug': False}]}, 'test_cfg': {'rpn': None, 'rcnn': {'max_per_img': 300, 'mask_thr_binary': 0.5, 'nms': {'type': 'nms', 'iou_threshold': 0.7}}}}, 'total_epochs': 36, 'min_values': (480, 512, 544, 576, 608, 640, 672, 704, 736, 768, 800), 'work_dir': './work_dirs/queryinst_r50_fpn_300_proposals_crop_mstrain_480-800_3x_coco', 'gpu_ids': range(0, 1), 'seed': None}

In [10]:
for key,value in cfg1.items():
    print('{key}:{value}'.format(key = key, value = value))

dataset_type:CocoDataset
data_root:data/coco/
img_norm_cfg:{'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}
train_pipeline:[{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'AutoAugment', 'policies': [[{'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}], [{'type': 'Resize', 'img_scale': [(400, 1333), (500, 1333), (600, 1333)], 'multiscale_mode': 'value', 'keep_ratio': True}, {'type': 'RandomCrop', 'crop_type': 'absolute_range', 'crop_size': (384, 600), 'allow_negative_crop': True}, {'type': 'Resize', 'img_scale': [(480, 1333), (512, 1333), (544, 1333), (576, 1333), (608, 1333), (640, 1333), (672, 1333), (704, 1333), (736, 1333), (768, 1333), (800, 1333)], 'multiscale_mode': 'value'

In [7]:
cfg.data.train

AttributeError: 'dict' object has no attribute 'data'

In [9]:
cfg['data']['train']

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_train2017.json',
 'img_prefix': 'data/coco/train2017/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
  {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size_divisor': 32},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]}

In [10]:
cfg1['data']['train']

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_train2017.json',
 'img_prefix': 'data/coco/train2017/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'AutoAugment',
   'policies': [[{'type': 'Resize',
      'img_scale': [(480, 1333),
       (512, 1333),
       (544, 1333),
       (576, 1333),
       (608, 1333),
       (640, 1333),
       (672, 1333),
       (704, 1333),
       (736, 1333),
       (768, 1333),
       (800, 1333)],
      'multiscale_mode': 'value',
      'keep_ratio': True}],
    [{'type': 'Resize',
      'img_scale': [(400, 1333), (500, 1333), (600, 1333)],
      'multiscale_mode': 'value',
      'keep_ratio': True},
     {'type': 'RandomCrop',
      'crop_type': 'absolute_range',
      'crop_size': (384, 600),
      'allow_negative_crop': True},
     {'type': 'Resize',
      'img_scale': [(480, 1333),
       (512, 1333),
   

In [14]:
def build_from_cfg(cfg, registry, default_args=None):
    """Build a module from config dict.

    Args:
        cfg (dict): Config dict. It should at least contain the key "type".
        registry (:obj:`Registry`): The registry to search the type from.
        default_args (dict, optional): Default initialization arguments.

    Returns:
        object: The constructed object.
    """
    if not isinstance(cfg, dict):
        raise TypeError(f'cfg must be a dict, but got {type(cfg)}')
    if 'type' not in cfg:
        if default_args is None or 'type' not in default_args:
            raise KeyError(
                '`cfg` or `default_args` must contain the key "type", '
                f'but got {cfg}\n{default_args}')

    if not (isinstance(default_args, dict) or default_args is None):
        raise TypeError('default_args must be a dict or None, '
                        f'but got {type(default_args)}')

    args = cfg.copy()

    if default_args is not None:
        for name, value in default_args.items():
            args.setdefault(name, value)

    return args

In [15]:
build_from_cfg(cfg['data']['train'], registry=None)

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_train2017.json',
 'img_prefix': 'data/coco/train2017/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
  {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'Pad', 'size_divisor': 32},
  {'type': 'DefaultFormatBundle'},
  {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]}

In [16]:
build_from_cfg(cfg1['data']['train'], registry=None)

{'type': 'CocoDataset',
 'ann_file': 'data/coco/annotations/instances_train2017.json',
 'img_prefix': 'data/coco/train2017/',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True},
  {'type': 'RandomFlip', 'flip_ratio': 0.5},
  {'type': 'AutoAugment',
   'policies': [[{'type': 'Resize',
      'img_scale': [(480, 1333),
       (512, 1333),
       (544, 1333),
       (576, 1333),
       (608, 1333),
       (640, 1333),
       (672, 1333),
       (704, 1333),
       (736, 1333),
       (768, 1333),
       (800, 1333)],
      'multiscale_mode': 'value',
      'keep_ratio': True}],
    [{'type': 'Resize',
      'img_scale': [(400, 1333), (500, 1333), (600, 1333)],
      'multiscale_mode': 'value',
      'keep_ratio': True},
     {'type': 'RandomCrop',
      'crop_type': 'absolute_range',
      'crop_size': (384, 600),
      'allow_negative_crop': True},
     {'type': 'Resize',
      'img_scale': [(480, 1333),
       (512, 1333),
   

In [17]:
type(build_from_cfg(cfg1['data']['train'], registry=None))

dict

In [21]:
tuple1 = tuple((1, 2, 3, 4))

In [22]:
print(tuple1)

(1, 2, 3, 4)


In [23]:
len(tuple1)

4

In [24]:
tensor1 = torch.rand((2,3))

In [25]:
str(tensor1.shape)

'torch.Size([2, 3])'

In [26]:
print(tensor1)

tensor([[0.8155, 0.0928, 0.2000],
        [0.5605, 0.5539, 0.8906]])


In [27]:
np1 = np.asarray([[[ 800., 1067.,  800., 1067.]],[[ 800., 1174.,  800., 1174.]]])

In [29]:
np1.shape

(2, 1, 4)

In [30]:
np1

array([[[ 800., 1067.,  800., 1067.]],

       [[ 800., 1174.,  800., 1174.]]])

In [31]:
np1[0]

array([[ 800., 1067.,  800., 1067.]])

In [32]:
target_lvls = torch.rand(5)
target_lvls

tensor([0.1921, 0.0380, 0.7934, 0.4240, 0.6503])

In [33]:
mask = target_lvls < 0.5
mask

tensor([ True,  True, False,  True, False])

In [34]:
inds = mask.nonzero(as_tuple=False).squeeze(1)
inds

tensor([0, 1, 3])

In [35]:
rois = torch.rand((5, 2))
rois

tensor([[0.6279, 0.6949],
        [0.4297, 0.3720],
        [0.1383, 0.0598],
        [0.8139, 0.2525],
        [0.6792, 0.7101]])

In [36]:
rois_ = rois[inds]
rois_

tensor([[0.6279, 0.6949],
        [0.4297, 0.3720],
        [0.8139, 0.2525]])

In [6]:
dict1 = {'bboxes': array([[  8.21, 193.18, 316.27, 605.29],
       [  0.  ,   0.  , 476.32, 482.93]], dtype=float32), 'labels': array([15,  1]), 'bboxes_ignore': array([], shape=(0, 4), dtype=float32), 'masks': [[[28.75, 540.94, 39.7, 456.05, 27.38, 404.02, 34.23, 343.78, 42.44, 323.25, 27.38, 278.07, 12.32, 224.67, 8.21, 208.24, 54.77, 217.82, 115.01, 215.09, 146.5, 193.18, 171.14, 200.02, 153.34, 236.99, 167.03, 250.68, 158.82, 267.11, 147.87, 295.86, 231.38, 315.03, 268.35, 350.63, 299.84, 382.12, 306.68, 416.35, 316.27, 467.0, 309.42, 523.14, 292.99, 536.83, 295.73, 556.0, 295.73, 579.27, 217.69, 605.29, 123.22, 605.29, 109.53, 592.96, 142.39, 577.9, 139.65, 534.09, 88.99, 519.03, 67.09, 565.58, 21.91, 551.89]], [[324.76, 432.3, 409.92, 351.47, 459.0, 241.77, 470.55, 178.26, 476.32, 98.87, 461.89, 36.81, 439.03, 0.0, 290.61, 0.0, 275.18, 0.0, 160.27, 0.0, 53.87, 0.0, 0.0, 0.0, 0.0, 106.15, 0.0, 116.79, 0.0, 116.79, 1.74, 122.11, 5.01, 472.73, 43.23, 482.93, 38.13, 377.18, 43.23, 332.58, 25.39, 270.15, 8.83, 224.28, 25.39, 220.46, 52.15, 221.74, 99.29, 215.37, 142.61, 207.72, 161.72, 198.8, 168.09, 215.37, 170.64, 265.06, 169.36, 280.34, 174.46, 293.09, 222.88, 300.73, 254.73, 326.21, 280.21, 379.72, 296.77, 411.58]]], 'seg_map': '000000525851.png'}

NameError: name 'array' is not defined

In [7]:
seg = [[200.69, 130.08, 199.66, 138.81, 199.66, 143.42, 209.92, 141.88, 215.56, 140.34, 215.56, 131.11, 220.18, 129.57, 222.23, 125.98, 222.23, 125.47, 224.8, 122.9, 230.95, 129.06, 236.6, 139.83, 235.57, 151.12, 240.19, 152.66, 242.75, 140.86, 238.13, 130.08, 227.87, 120.34, 228.9, 114.69, 239.16, 117.26, 236.08, 108.54, 230.95, 102.38, 224.8, 95.71, 207.87, 94.69, 196.07, 99.82, 192.99, 109.05, 190.94, 115.21, 182.73, 127.52, 178.63, 132.65, 184.27, 133.67, 188.37, 127.52, 191.96, 123.93, 196.58, 121.88, 198.12, 123.41, 199.15, 129.57, 199.66, 129.57]]

In [12]:
np.array(seg).shape

(1, 70)

In [13]:
rle = [1, 2, 3]
sum(rle)

6

In [14]:
ref_bboxes = torch.rand((2, 3, 4))

In [15]:
ref_bboxes[0].size(0)

3

In [11]:
pos_bboxes = torch.rand((4, 4))
neg_bboxes = torch.rand((2, 4))

In [12]:
res = torch.cat([pos_bboxes, neg_bboxes])

In [13]:
res.shape

torch.Size([6, 4])

In [7]:
overlaps = torch.rand((3, 4))
num_bboxes = 4
overlaps.new_full((num_bboxes, ), -1, dtype=torch.long).shape

torch.Size([4])

In [8]:
overlaps.new_full((num_bboxes, ), -1, dtype=torch.long)

tensor([-1, -1, -1, -1])

In [9]:
overlaps

tensor([[0.8525, 0.8651, 0.4811, 0.7974],
        [0.1863, 0.8352, 0.3759, 0.7584],
        [0.9045, 0.3167, 0.7406, 0.5487]])

In [10]:
max_overlaps, argmax_overlaps = overlaps.max(dim=0) 

In [11]:
max_overlaps

tensor([0.9045, 0.8651, 0.7406, 0.7974])

In [12]:
argmax_overlaps

tensor([2, 0, 2, 0])

In [13]:
assigned_gt_inds = overlaps.new_full((num_bboxes, ), -1, dtype=torch.long)

In [14]:
assigned_gt_inds

tensor([-1, -1, -1, -1])

In [16]:
assigned_gt_inds[(max_overlaps >= 0)& (max_overlaps < 0.8)] = 0

In [17]:
assigned_gt_inds

tensor([-1, -1,  0,  0])

In [18]:
pos_inds = max_overlaps >= 0.8
pos_inds

tensor([ True,  True, False, False])

In [19]:
assigned_gt_inds[pos_inds] = argmax_overlaps[pos_inds] + 1 
assigned_gt_inds

tensor([3, 1, 0, 0])

In [20]:
gt_max_overlaps, gt_argmax_overlaps = overlaps.max(dim=1)

In [23]:
gt_max_overlaps

tensor([0.8651, 0.8352, 0.9045])

In [22]:
gt_argmax_overlaps

tensor([1, 1, 0])

In [25]:
a = [1, 2, 3]
a.index(1)

0

In [26]:
a.index(2)

1

In [27]:
a.index(3)

2

In [29]:
torch.nonzero(gt_max_overlaps > 0).squeeze(1)

tensor([0, 1, 2])

In [32]:
a = torch.rand((3))
b = torch.rand((3))

In [33]:
torch.cat([a, b],0).shape

torch.Size([6])